In [1]:
import glob2 as glob
from PIL import Image, ImageDraw, ImageFont
import matplotlib.pyplot as plt
import numpy as np

# Import NGDLM models.
!pip install git+https://github.com/AI-Guru/ngdlm.git
from ngdlm import models as ngdlmodels

  Cloning https://github.com/AI-Guru/ngdlm.git to /tmp/pip-req-build-d634hs5s
  Running setup.py bdist_wheel for ngdlm ... done
  Stored in directory: /tmp/pip-ephem-wheel-cache-hgzug5et/wheels/93/06/27/e156acb49f475c364c3c9fa4ad4ab7bfa38808bff5bf9c4647
Successfully built ngdlm


Using TensorFlow backend.


In [4]:
from keras import models, layers, initializers
from ngdlm import models as ngdlmodels
from ngdlm import utils as ngdlutils
from keras import models, layers
from keras import initializers


In [11]:
latent_dim = 10

# Encoder.
encoder_input = layers.Input(shape=(128, 128,1))
encoder_output = layers.Conv2D(32, (3, 3),
                               kernel_initializer=initializers.lecun_normal(seed=0),
                               activation="relu",
                               padding="same")(encoder_input)
encoder_output = layers.MaxPooling2D((2, 2),padding="same")(encoder_output)
encoder_output = layers.Conv2D(32, (3, 3),
                               kernel_initializer=initializers.lecun_normal(seed=0),
                               activation="relu",
                               padding="same")(encoder_output)
encoder_output = layers.MaxPooling2D((2, 2),padding="same")(encoder_output)
encoder_output = layers.Flatten()(encoder_output)
encoder_output = layers.Dense(1024,
                              activation = "relu",
                              kernel_initializer=initializers.lecun_normal(seed=0)
                             )(encoder_output)
encoder_output = layers.Dense(64,
                              activation = "relu",
                              kernel_initializer=initializers.lecun_normal(seed=0)
                             )(encoder_output)
encoder = models.Model(encoder_input, encoder_output)

# Decoder.
decoder_input = layers.Input(shape=(latent_dim,))
decoder_output = layers.Dense(64,
                              activation="relu",
                              kernel_initializer=initializers.lecun_normal(seed=0)
                             )(decoder_input)
decoder_output = layers.Dense(1024,
                              activation="sigmoid",
                              kernel_initializer=initializers.lecun_normal(seed=0)
                             )(decoder_output)
decoder_output = layers.Dense(32*32*32,
                              activation="sigmoid",
                              kernel_initializer=initializers.lecun_normal(seed=0)
                             )(decoder_output)
decoder_output = layers.Reshape((32, 32, 32))(decoder_output)
decoder_output = layers.Conv2D(32, (3, 3),
                               kernel_initializer=initializers.lecun_normal(seed=0),
                               activation="relu",
                               padding="same")(decoder_output)
decoder_output = layers.UpSampling2D((2, 2))(decoder_output)
decoder_output = layers.Conv2D(32, (3, 3),
                               kernel_initializer=initializers.lecun_normal(seed=0),
                               activation="relu",
                               padding="same")(decoder_output)
decoder_output = layers.UpSampling2D((2, 2))(decoder_output)
decoder_output = layers.Conv2D(1, (3, 3),
                               kernel_initializer=initializers.lecun_normal(seed=0),
                               activation="sigmoid",
                               padding="same")(decoder_output)
decoder = models.Model(decoder_input, decoder_output)

# Autoencoder.
autoencoder = ngdlmodels.VAE(encoder, decoder, latent_dim=latent_dim)    #glue encoder and decoder together
autoencoder.compile(optimizer="adadelta", loss="binary_crossentropy")
autoencoder.summary()

Encoder:
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_8 (InputLayer)            (None, 128, 128, 1)  0                                            
__________________________________________________________________________________________________
conv2d_11 (Conv2D)              (None, 128, 128, 32) 320         input_8[0][0]                    
__________________________________________________________________________________________________
max_pooling2d_7 (MaxPooling2D)  (None, 64, 64, 32)   0           conv2d_11[0][0]                  
__________________________________________________________________________________________________
conv2d_12 (Conv2D)              (None, 64, 64, 32)   9248        max_pooling2d_7[0][0]            
__________________________________________________________________________________________________
m

In [22]:
# Train- and validation-data.
from PIL import Image
import glob2 as glob
from aux_functions import helpers

image_list = []
for filename in glob.glob('../data/a/*.png'): #assuming gif
    im=np.array(Image.open(filename))
    im=helpers.rgb2gray(im)
    image_list.append(im)

In [31]:
import pickle
pickle.dump(image_list, open("../data/pickled_a.p","wb"))

In [19]:
x_input_train = np.array(image_list)
#(x_train, _), (x_test, _)= mnist.load_data()

x_input_train = x_input_train.astype('float32')/255.0
#x_input_validate = x_test.astype('float32')/255.0 

x_input_train = x_input_train.reshape((-1, 128, 128))

In [20]:
x_input_train.shape

(369, 128, 128)

In [21]:
# Train.
print("Train...")
history = autoencoder.fit(
        x_input_train, x_input_train,
        epochs=10,
        batch_size=1,
        shuffle=True
        #validation_data=(x_input_validate, x_input_validate)
    )

# Evaluate.
#print("Evaluate...")
#loss = autoencoder.evaluate(x_input_test, x_input_test)
#print("Loss:", loss)

Train...
Epoch 1/10
127/369 [=========>....................] - ETA: 49s - loss: 2.4773

KeyboardInterrupt: 

In [ ]:
decoded_imgs = autoencoder.predict(x_input_train) 

In [ ]:
print(decoded_imgs.shape)

In [ ]:
n = 10
plt.figure(figsize=(20,4))
for i in range(n):
    ax = plt.subplot(2, n, i + 1)
    plt.imshow(x_input_train[i].reshape((128,128)), cmap = "gray")
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
    
    ax = plt.subplot(2, n, i + 1 + n)
    plt.imshow(decoded_imgs[i].reshape((128, 128)), cmap = "gray")
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

plt.show()
plt.close()